In [1]:
import nibabel as nib
import numpy as np
from pathlib import Path

In [6]:
def scale_tmap_by_max(tmap_nifti):
    # Load the NIfTI file
    tmap_img = nib.load(tmap_nifti)
    
    # Get the data from the NIfTI file
    tmap_data = tmap_img.get_fdata()
    
    # Scale the t-map by its maximum value
    max_value = np.max(tmap_data)
    scaled_tmap_data = tmap_data / max_value
    
    # Create a new NIfTI image with the scaled data
    scaled_tmap_img = nib.Nifti1Image(scaled_tmap_data, tmap_img.affine, tmap_img.header)
    nib.save(scaled_tmap_img, tmap_nifti.parent / f"{tmap_nifti.stem}_peak-scaled.nii.gz")
    return scaled_tmap_img



In [7]:
results_dir = Path('/home/neel/Documents/SPM_results/second_level')
for contrast_dir in results_dir.iterdir():
    if contrast_dir.is_dir():
        print(f"Processing contrast: {contrast_dir.name}")
        tmap_file = contrast_dir / "spmT_0001.nii"
        scaled_tmap_img = scale_tmap_by_max(tmap_file)

Processing contrast: SPM-A_II_syllables_IPA_eSpeak (FINAL)
Processing contrast: SPM-A_II_multireg_test (FINAL)
Processing contrast: SPM-V_II_Zipf_multireg_december_allsubs (FINAL)
Processing contrast: SPM-V_II_Lg10BG_multireg_november


Function to split each tmap into L/R hemispheric halves

In [4]:
import nibabel as nib
import numpy as np
from pathlib import Path

def split_tmap_left_right(tmap_file, left_output="left_tmap.nii.gz", right_output="right_tmap.nii.gz"):
    """
    Loads a T-map in MNI space and splits it into two volumes:
      1) left_tmap.nii.gz: contains T-values where x < 0
      2) right_tmap.nii.gz: contains T-values where x > 0
    
    Voxels outside the chosen hemisphere are zeroed out.
    Voxels exactly on x=0 are ignored (set to zero) by default.
    """
    #--------------------------------------------------------------------------
    # 1) Load the T-map (in MNI space)
    #--------------------------------------------------------------------------
    tmap_img = nib.load(tmap_file)
    tmap_data = tmap_img.get_fdata()
    affine = tmap_img.affine
    header = tmap_img.header  # preserve header if desired

    #--------------------------------------------------------------------------
    # 2) Determine the real-world (MNI) x-coordinates of each voxel
    #--------------------------------------------------------------------------
    dims = tmap_data.shape
    i, j, k = np.meshgrid(
        np.arange(dims[0]),
        np.arange(dims[1]),
        np.arange(dims[2]),
        indexing="ij"
    )

    voxel_coords = np.column_stack((i.flatten(), j.flatten(), k.flatten()))
    mni_coords = nib.affines.apply_affine(affine, voxel_coords)

    # x-coordinates in MNI space
    x_coords = mni_coords[:, 0]

    #--------------------------------------------------------------------------
    # 3) Create arrays to hold left- and right-hemisphere T-values
    #--------------------------------------------------------------------------
    left_tmap_data = np.zeros(dims, dtype=tmap_data.dtype)
    right_tmap_data = np.zeros(dims, dtype=tmap_data.dtype)

    # Masks for x < 0 (left) and x > 0 (right)
    left_mask = x_coords < 0
    right_mask = x_coords > 0

    #--------------------------------------------------------------------------
    # 4) Assign T-values to left/right volumes
    #--------------------------------------------------------------------------
    # Left hemisphere
    left_indices = voxel_coords[left_mask]
    left_tmap_data[left_indices[:, 0],
                   left_indices[:, 1],
                   left_indices[:, 2]] = tmap_data[left_indices[:, 0],
                                                  left_indices[:, 1],
                                                  left_indices[:, 2]]

    # Right hemisphere
    right_indices = voxel_coords[right_mask]
    right_tmap_data[right_indices[:, 0],
                    right_indices[:, 1],
                    right_indices[:, 2]] = tmap_data[right_indices[:, 0],
                                                     right_indices[:, 1],
                                                     right_indices[:, 2]]

    #--------------------------------------------------------------------------
    # 5) Save out NIfTI files for left/right T-maps
    #--------------------------------------------------------------------------
    left_img = nib.Nifti1Image(left_tmap_data, affine, header=header)
    right_img = nib.Nifti1Image(right_tmap_data, affine, header=header)


    dir = Path(tmap_file).parent
    left_output = dir / left_output
    right_output = dir / right_output
    nib.save(left_img, str(left_output))
    nib.save(right_img, str(right_output))

    print(f"Saved left hemisphere T-map to: {left_output}")
    print(f"Saved right hemisphere T-map to: {right_output}")

In [5]:
results_dir = Path('/home/neel/Documents/SPM_results/second_level')
for contrast_dir in results_dir.iterdir():
    if contrast_dir.is_dir():
        print(f"Processing contrast: {contrast_dir.name}")
        tmap_file = contrast_dir / "spmT_0001.nii"
        split_tmap_left_right(tmap_file, left_output="left_tmap.nii.gz", right_output="right_tmap.nii.gz")

Processing contrast: SPM-A_II_syllables_IPA_eSpeak (FINAL)
Saved left hemisphere T-map to: /home/neel/Documents/SPM_results/second_level/SPM-A_II_syllables_IPA_eSpeak (FINAL)/left_tmap.nii.gz
Saved right hemisphere T-map to: /home/neel/Documents/SPM_results/second_level/SPM-A_II_syllables_IPA_eSpeak (FINAL)/right_tmap.nii.gz
Processing contrast: SPM-A_II_multireg_test (FINAL)
Saved left hemisphere T-map to: /home/neel/Documents/SPM_results/second_level/SPM-A_II_multireg_test (FINAL)/left_tmap.nii.gz
Saved right hemisphere T-map to: /home/neel/Documents/SPM_results/second_level/SPM-A_II_multireg_test (FINAL)/right_tmap.nii.gz
Processing contrast: SPM-V_II_Zipf_multireg_december_allsubs (FINAL)
Saved left hemisphere T-map to: /home/neel/Documents/SPM_results/second_level/SPM-V_II_Zipf_multireg_december_allsubs (FINAL)/left_tmap.nii.gz
Saved right hemisphere T-map to: /home/neel/Documents/SPM_results/second_level/SPM-V_II_Zipf_multireg_december_allsubs (FINAL)/right_tmap.nii.gz
Processing